In [22]:
import os
from langchain.output_parsers.openai_tools import JsonOutputToolsParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableLambda
from langchain.chains import create_extraction_chain
from typing import Optional, List
from langchain.chains import create_extraction_chain_pydantic
from langchain_core.pydantic_v1 import BaseModel
from langchain import hub
from langchain.docstore.document import Document
from cerebras.cloud.sdk import Cerebras
from langsmith import Client
import json
from langchain.docstore.document import Document
from dotenv import load_dotenv
# from rich import print
from langchain.docstore.document import Document
from langchain_community.chat_models import ChatOllama
from langchain_community.vectorstores import Chroma
from langchain_community import embeddings
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
import torch
from logger import logger
import os
import json
import uuid
import numpy as np
import torch
import faiss
from dotenv import load_dotenv
from typing import Optional
from sentence_transformers import SentenceTransformer, CrossEncoder
from rank_bm25 import BM25Okapi
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.docstore.document import Document
from cerebras.cloud.sdk import Cerebras
import pandas as pd
import ast

# Load environment variables
load_dotenv()

# Use GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

Using device: cuda


# Load API Keys of Cerebras

In [23]:
# Load all available API keys
api_keys = [os.getenv(f"CEREBRAS_API_KEY_{i}") for i in range(7)]

api_keys = [key for key in api_keys if key]  # Remove None values
print(f"Found {len(api_keys)} Cerebras API keys.")

# print(api_keys)
if not api_keys:
    raise ValueError("No Cerebras API keys found in environment variables")

current_api_key_index = 0
client = Cerebras(api_key=api_keys[current_api_key_index])
model = "gpt-oss-120b"
print(api_keys)
# prompt = "Hello, Cerebras!"
# response = client.chat.completions.create(
#                 messages=[{"role": "user", "content": prompt}],
#                 model=model,
#             )
# print(response.choices[0].message.content)

Found 6 Cerebras API keys.
[26/Aug/2025 18:26:12] INFO - HTTP Request: GET https://api.cerebras.ai/v1/tcp_warming "HTTP/1.1 200 OK"
['csk-x2kewhvre2y3cvt859wckjvh66wcn3n3k3nwcn5n4kem8exc', 'csk-pjfmp5jn93rmvmn2hcprvfjd5ck59fcfxevtpd3c4ycvvktx', 'csk-n328539mn5h84f22jyw9n3r2vvwypd6hwy8262wmkftnke4f', 'csk-nhh664njyv366pvyyn8v8h42jhncxkhw5mvmk8pdvt5yfdvk', 'csk-edvj62mc95848jm8dwmp3e2kkc2dn5xmyx4t8n8c32ntec8w', 'csk-24tkwkcyjfjy4vmynjw6j45d3m53rvjpw2chmyfwwk5fytm8']


# Create Propositions for the Knowledge Base chunking

In [24]:
def cerebras_invoke(prompt: str) -> str:
    global client, current_api_key_index, api_keys
    while True:
        try:
            # raise Exception("request_quota_exceeded: simulated error")
            response = client.chat.completions.create(
                messages=[{"role": "user", "content": prompt}],
                model=model,
                temperature=0.4
            )
            return response.choices[0].message.content.strip()
        except Exception as e:
            error_msg = e.__str__()
            if "request_quota_exceeded" in error_msg:
                logger.error(f"Request quota exceeded for key serial {current_api_key_index}.")
                current_api_key_index += 1
                logger.info(f"Switching to API key serial {current_api_key_index}.")
            elif "token_quota_exceeded" in error_msg:
                logger.error(f"Token quota exceeded for key serial {current_api_key_index}. Please check your API key limits.")
                api_keys.pop(current_api_key_index)
            else:
                logger.info(f"Switching to API key serial {current_api_key_index} due to an error.")
                logger.error(f"An error occurred: {error_msg}")
                raise e

            if current_api_key_index >= len(api_keys):
                current_api_key_index = 0
            if len(api_keys) == 0:
                raise ValueError("All Cerebras API keys have exceeded their quotas or are invalid.")
            client = Cerebras(api_key=api_keys[current_api_key_index])

In [4]:
# Pull the proposal-indexing prompt from the hub
LANGSMITH_API_KEY = os.getenv("LANGSMITH_API_KEY")
obj = Client(api_key=LANGSMITH_API_KEY).pull_prompt("wfh/proposal-indexing", include_model=True)
class Sentences(BaseModel):
    sentences: List[str]

In [25]:
def get_propositions(text, prompt):
    formatted_prompt = prompt.format(input=text) + "\n\nOnly provide the list of propositions as output. Do not include any explanations, formatting, or additional text."
    # print(f"Formatted Prompt: {formatted_prompt}")
    response = cerebras_invoke(formatted_prompt)
    # print(f"Response: {response}")
    propositions = response.split('\n')
    return {"proposition": [Sentences(sentences=propositions)]}, response

In [51]:
# Define checkpoint path
checkpoint_dir = "Propositions"
checkpoint_path = os.path.join(checkpoint_dir, "proposition_indexing_checkpoint.json")
# Create directory if not exists
os.makedirs(checkpoint_dir, exist_ok=True)

if os.path.exists(checkpoint_path):
    with open(checkpoint_path, "r") as f:
        proposition_arrays = json.load(f)
else:
    proposition_arrays = []

# Save the checkpoint function
def save_checkpoint(data):
    with open(checkpoint_path, "w") as f:
        json.dump(data, f, indent=4)
    print(f"Checkpoint saved to {checkpoint_path}")
# Load JSON data
with open("islamic_etiquette_knowledge_base.json", "r") as f1, open("Quran_app_Documentation.json", "r") as f2:
    etiquette_data = json.load(f1)
    quran_app_data = json.load(f2)

# Use only Quran app data as per the query only first 3 documents
combined_documents = quran_app_data + etiquette_data 

# Track already processed docs (using URL as unique ID)
processed_urls = {doc["metadata"]["url"] for doc in proposition_arrays}

# Process each JSON object
for i,json_obj in enumerate(combined_documents):
    if json_obj["url"] in processed_urls:
        print(f"Skipping already processed: {json_obj['url']}")
        continue  # skip docs already in checkpoint

    text = json_obj['text']
    propositions, response = get_propositions(text, obj)
    
    document_propositions = {
        'metadata': {
            'url': json_obj['url'],
            'title': json_obj['title']
        },
        'propositions': [
            prop for prop in propositions['proposition'][0].sentences if prop.strip()
        ]
    }
    proposition_arrays.append(document_propositions)
    save_checkpoint(proposition_arrays)
    #  # 💥 Force crash after processing first document
    # if i == 0:
    #     raise RuntimeError("Simulated crash!")

# Flatten after loop finishes
flat_propositions_with_metadata = [
    {"proposition": prop, "metadata": doc["metadata"]}
    for doc in proposition_arrays
    for prop in doc["propositions"]
]

propositions_list = [prop for doc in proposition_arrays for prop in doc["propositions"]]


Skipping already processed: https://gtaf.org/blog/audio-feature-in-quran-app/?fbclid=IwY2xjawL0bIZleHRuA2FlbQIxMABicmlkETFmSGVJS3hCRDlCMU5zNUhqAR4aYJYn7ftHGkfYQbTiZDuaCaRNOz1AspTU0loZZgmK3EcN74sWyhMeXLNOhw_aem_oNi9qtfPwjVOyVVq36IE3g
Skipping already processed: https://gtaf.org/blog/search-option-in-quran-app/?fbclid=IwY2xjawL0bIJleHRuA2FlbQIxMABicmlkETFmSGVJS3hCRDlCMU5zNUhqAR7nLIJEj0oMqn073aQlqYn2PNe5XmAIqD1abGPapvnOE0qxxRS7wulIb7JGIg_aem_8JhZJyueZ1bw6-qnBZydmA
Skipping already processed: https://gtaf.org/blog/planner-feature-in-quran-app/?fbclid=IwY2xjawL0bIxleHRuA2FlbQIxMABicmlkETFmSGVJS3hCRDlCMU5zNUhqAR5qbQoNUXlMpzbxIshcw3YRhMhM0UKRfkfXXEzFxsoyxt-y1b-jJeylEILGQQ_aem_P6GPwvd1bq_B3fTNbMNSUg
Skipping already processed: https://gtaf.org/blog/explore-by-topic-in-quran-app/?fbclid=IwY2xjawL0bHxleHRuA2FlbQIxMABicmlkETFmSGVJS3hCRDlCMU5zNUhqAR4oaAxZ-_-1W9pdsQWXvY5qQYbtz6Ti2-byIf6H_WMBG2dXZ-SpAWfi-xsuIQ_aem_CgtPkPGc61w3zh6gIBFt1Q
Skipping already processed: https://gtaf.org/blog/build-a-daily

In [52]:
print(len(propositions_list))
print("Current API Key Index:", current_api_key_index)

702
Current API Key Index: 0


# Agentic Chunker Class

In [53]:
class AgenticChunker:
    def __init__(self, client=None, model="gpt-oss-120b", checkpoint_path=None):
        self.chunks = {}
        self.id_truncate_limit = 5
        self.generate_new_metadata_ind = True
        self.print_logging = True
        self.client = client
        self.model = model

        # If no client provided, fall back to env var
        if self.client is None:
            cerebras_api_key = os.getenv("CEREBRAS_API_KEY")
            if cerebras_api_key is None:
                raise ValueError("Cerebras client not provided and no CEREBRAS_API_KEY found.")
            self.client = Cerebras(api_key=cerebras_api_key)

        # Checkpoint directory
        checkpoint_dir = "agentic_chunks"
        os.makedirs(checkpoint_dir, exist_ok=True)

        self.checkpoint_path = checkpoint_path or os.path.join(
            checkpoint_dir, "agentic_chunks_checkpoint.json"
        )

        # Load checkpoint if exists
        if os.path.exists(self.checkpoint_path):
            self._load_checkpoint()

    def _save_checkpoint(self):
        with open(self.checkpoint_path, "w") as f:
            json.dump(self.chunks, f, indent=4)
        if self.print_logging:
            print(f"[Checkpoint] Saved chunks to {self.checkpoint_path}")

    def _load_checkpoint(self):
        with open(self.checkpoint_path, "r") as f:
            self.chunks = json.load(f)
        if self.print_logging:
            print(f"[Checkpoint] Loaded chunks from {self.checkpoint_path}")

    def _llm_invoke(self, prompt: str) -> str:
        global client, current_api_key_index, api_keys
        while True:
            try:
                print("Api key index:", current_api_key_index)
                print("Api key:", api_keys[current_api_key_index])
                response = client.chat.completions.create(
                    messages=[{"role": "user", "content": prompt}],
                    model=model,
                    temperature=0.4
                )
                return response.choices[0].message.content.strip()
            except Exception as e:
                error_msg = e.__str__()
                if "request_quota_exceeded" in error_msg:
                    logger.error(f"Request quota exceeded for key serial {current_api_key_index}.")
                    current_api_key_index += 1
                    logger.info(f"Switching to API key serial {current_api_key_index}.")
                elif "token_quota_exceeded" in error_msg:
                    logger.error(f"Token quota exceeded for key serial {current_api_key_index}. Please check your API key limits.")
                    api_keys.pop(current_api_key_index)
                else:
                    logger.info(f"Switching to API key serial {current_api_key_index} due to an error.")
                    logger.error(f"An error occurred: {error_msg}")
                    raise e

                if current_api_key_index >= len(api_keys):
                    current_api_key_index = 0
                if len(api_keys) == 0:
                    raise ValueError("All Cerebras API keys have exceeded their quotas or are invalid.")
                client = Cerebras(api_key=api_keys[current_api_key_index])
    def add_propositions(self, propositions: list):
        for proposition in propositions:
            self.add_proposition(proposition)

    def add_proposition(self, proposition: str):
        if self.print_logging:
            print(f"\nAdding: '{proposition}'")
        if len(self.chunks) == 0:
            if self.print_logging:
                print("No chunks, creating a new one")
            self._create_new_chunk(proposition)
            self._save_checkpoint()
            return

        chunk_id = self._find_relevant_chunk(proposition)
        if chunk_id:
            if self.print_logging:
                print(f"Chunk Found ({self.chunks[chunk_id]['chunk_id']}), adding to: {self.chunks[chunk_id]['title']}")
            self.add_proposition_to_chunk(chunk_id, proposition)
        else:
            if self.print_logging:
                print("No chunks found")
            self._create_new_chunk(proposition)

        # ✅ Save after every update
        self._save_checkpoint()

    def add_proposition_to_chunk(self, chunk_id: str, proposition: str):
        self.chunks[chunk_id]['propositions'].append(proposition)
        if self.generate_new_metadata_ind:
            self.chunks[chunk_id]['summary'] = self._update_chunk_summary(self.chunks[chunk_id])
            self.chunks[chunk_id]['title'] = self._update_chunk_title(self.chunks[chunk_id])


    def _update_chunk_summary(self, chunk: dict) -> str:
        prompt = (
            "You are the steward of a group of chunks which represent groups of sentences that talk about a similar topic, with a focus on incorporating Islamic values and teachings.\n"
            "A new proposition was just added to one of your chunks. Generate a very brief 1-sentence summary which will inform viewers what a chunk group is about, and relate it to Islamic principles where possible.\n"
            "A good summary will say what the chunk is about, give any clarifying instructions on what to add to the chunk, and generalize appropriately. If you get a proposition about zakat, generalize it to 'Islamic charity'. If about Ramadan, generalize to 'Islamic months and practices'.\n"
            "Examples:\n"
            "Input: Proposition: Muslims are encouraged to give zakat to help those in need\n"
            "Output: This chunk contains information about Islamic charity and the importance of zakat in supporting the needy.\n"
            "Input: Proposition: Fasting during Ramadan is obligatory for adult Muslims\n"
            "Output: This chunk discusses Islamic practices related to fasting and the significance of Ramadan.\n"
            "Only respond with the chunk's new summary, nothing else.\n"
            f"Chunk's propositions:\n" + "\n".join(chunk['propositions']) +
            f"\n\nCurrent chunk summary:\n{chunk['summary']}"
        )
        return self._llm_invoke(prompt)


    def _update_chunk_title(self, chunk: dict) -> str:
        prompt = (
            "You are the steward of a group of chunks which represent groups of sentences that talk about a similar topic, with a focus on incorporating Islamic values, culture, and teachings.\n"
            "A new proposition was just added to one of your chunks. Generate a very brief, updated chunk title (a few words) which will inform viewers what the chunk group is about, and relate it to Islamic principles or generalize appropriately.\n"
            "A good title will say what the chunk is about, and generalize where possible. For example, if you get a proposition about zakat, generalize it to 'Islamic charity'. If about Ramadan, generalize to 'Islamic months and practices'.\n"
            "You will be given a group of propositions in the chunk, the chunk summary, and the current chunk title.\n"
            "Your title should anticipate generalization and reflect Islamic context.\n"
            "Examples:\n"
            "Input: Summary: This chunk contains information about Islamic charity and the importance of zakat in supporting the needy\n"
            "Output: Islamic Charity & Zakat\n"
            "Input: Summary: This chunk discusses Islamic practices related to fasting and the significance of Ramadan\n"
            "Output: Ramadan & Fasting Practices\n"
            "Only respond with the new chunk title, nothing else.\n"
            f"Chunk's propositions:\n" + "\n".join(chunk['propositions']) +
            f"\n\nChunk summary:\n{chunk['summary']}\n\nCurrent chunk title:\n{chunk['title']}"
        )
        return self._llm_invoke(prompt)



    def _get_new_chunk_summary(self, proposition: str) -> str:
        prompt = (
            "You are the steward of a group of chunks which represent groups of sentences that talk about a similar topic, with a focus on incorporating Islamic values, culture, and teachings.\n"
            "You should generate a very brief 1-sentence summary which will inform viewers what a chunk group is about, and relate it to Islamic principles or generalize appropriately.\n"
            "A good summary will say what the chunk is about, give any clarifying instructions on what to add to the chunk, and generalize where possible. For example, if you get a proposition about zakat, generalize it to 'Islamic charity'. If about Ramadan, generalize to 'Islamic months and practices'.\n"
            "You will be given a proposition which will go into a new chunk. This new chunk needs a summary.\n"
            "Your summaries should anticipate generalization and reflect Islamic context.\n"
            "Examples:\n"
            "Input: Proposition: Muslims are encouraged to give zakat to help those in need\n"
            "Output: This chunk contains information about Islamic charity and the importance of zakat in supporting the needy.\n"
            f"Determine the summary of the new chunk that this proposition will go into:\n{proposition}"
        )
        return self._llm_invoke(prompt)



    def _get_new_chunk_title(self, summary: str) -> str:
        prompt = (
            "You are the steward of a group of chunks which represent groups of sentences that talk about a similar topic, with a focus on incorporating Islamic values, culture, and teachings.\n"
            "Your task is to generate a very brief chunk title (a few words) that clearly communicates what the chunk group is about, and relates it to Islamic principles or generalizes appropriately.\n"
            "A good chunk title is concise, generalizes where possible, and reflects Islamic context. For example, if the summary is about zakat, generalize to 'Islamic Charity'. If about Ramadan, generalize to 'Islamic Months & Practices'.\n"
            "You will be given a summary of a chunk which needs a title.\n"
            "Examples:\n"
            "Input: Summary: This chunk contains information about Islamic charity and the importance of zakat in supporting the needy.\n"
            "Output: Islamic Charity & Zakat\n"
            "Only respond with the new chunk title, nothing else.\n"
            f"Determine the title of the chunk that this summary belongs to:\n{summary}"
        )
        return self._llm_invoke(prompt)

    def _create_new_chunk(self, proposition: str):
        new_chunk_id = str(uuid.uuid4())[:self.id_truncate_limit]
        new_chunk_summary = self._get_new_chunk_summary(proposition)
        new_chunk_title = self._get_new_chunk_title(new_chunk_summary)
        self.chunks[new_chunk_id] = {
            'chunk_id': new_chunk_id,
            'propositions': [proposition],
            'title': new_chunk_title,
            'summary': new_chunk_summary,
            'chunk_index': len(self.chunks)
        }
        if self.print_logging:
            print(f"Created new chunk ({new_chunk_id}): {new_chunk_title}")

    def get_chunk_outline(self) -> str:
        chunk_outline = ""
        for chunk_id, chunk in self.chunks.items():
            single_chunk_string = f"""Chunk ({chunk['chunk_id']}): {chunk['title']}\nSummary: {chunk['summary']}\n\n"""
            chunk_outline += single_chunk_string
        return chunk_outline


    def _find_relevant_chunk(self, proposition: str) -> Optional[str]:
        current_chunk_outline = self.get_chunk_outline()
        prompt = (
            "Determine whether or not the 'Proposition' should belong to any of the existing chunks, with a focus on Islamic culture, values, and teachings.\n"
            "A proposition should belong to a chunk if their meaning, direction, or intention are similar, especially in the context of Islamic principles.\n"
            "The goal is to group similar propositions and chunks, generalizing to Islamic concepts where appropriate.\n"
            "If you think a proposition should be joined with a chunk, return the chunk id.\n"
            "If you do not think an item should be joined with an existing chunk, just return 'No chunks'.\n"
            "Example:\n"
            "Input:\n"
            "    - Proposition: 'Muslims are encouraged to give zakat to help those in need'\n"
            "    - Current Chunks:\n"
            "        - Chunk ID: 2n4l3d\n"
            "        - Chunk Name: Islamic Festivals\n"
            "        - Chunk Summary: Overview of important celebrations in Islam\n"
            "        - Chunk ID: 93833k\n"
            "        - Chunk Name: Islamic Charity & Zakat\n"
            "        - Chunk Summary: Information about charity and zakat in Islam\n"
            "Output: 93833k\n"
            f"Current Chunks:\n--Start of current chunks--\n{current_chunk_outline}\n--End of current chunks--\n"
            f"Determine if the following statement should belong to one of the chunks outlined:\n{proposition}\n"
            f"Do not write anything else. Only return the chunk id if you think it should belong to a chunk, or 'No chunks relevant to the proposition' if it should not.\n"
        )
        # print(f"\n[DEBUG] LLM Prompt:\n{prompt}\n")
        chunk_found = self._llm_invoke(prompt).strip()
        print(f"[DEBUG] Chunk Found: {chunk_found}")
        if len(chunk_found) == self.id_truncate_limit and chunk_found in self.chunks:
            return chunk_found
        return None

    def get_chunks(self, get_type: str = 'dict') -> list:
        if get_type == 'dict':
            return self.chunks
        if get_type == 'list_of_strings':
            return [" ".join(chunk['propositions']) for chunk in self.chunks.values()]

    def pretty_print_chunks(self):
        print(f"\nYou have {len(self.chunks)} chunks\n")
        for chunk_id, chunk in self.chunks.items():
            print(f"Chunk #{chunk['chunk_index']}")
            print(f"Chunk ID: {chunk_id}")
            print(f"Summary: {chunk['summary']}")
            print(f"Propositions:")
            for prop in chunk['propositions']:
                print(f"    - {prop}")
            print("\n")

    def pretty_print_chunk_outline(self):
        print("Chunk Outline\n")
        print(self.get_chunk_outline())

In [54]:
# Initialize AgenticChunker and add propositions
ac = AgenticChunker(client=client, model=model)
if os.path.exists(checkpoint_path):
    with open(checkpoint_path, "r") as f:
        proposition_arrays = json.load(f)

propositions_list = [prop for doc in proposition_arrays for prop in doc["propositions"]]

In [55]:
ac.add_propositions(propositions_list)


Adding: '["Audio is available to listen to the Quran.", '
No chunks, creating a new one
Api key index: 0
Api key: csk-x2kewhvre2y3cvt859wckjvh66wcn3n3k3nwcn5n4kem8exc
[23/Aug/2025 21:43:37] INFO - HTTP Request: POST https://api.cerebras.ai/v1/chat/completions "HTTP/1.1 200 OK"
Api key index: 0
Api key: csk-x2kewhvre2y3cvt859wckjvh66wcn3n3k3nwcn5n4kem8exc
[23/Aug/2025 21:43:37] INFO - HTTP Request: POST https://api.cerebras.ai/v1/chat/completions "HTTP/1.1 200 OK"
Created new chunk (7e8ab): Qur'an Audio Resources
[Checkpoint] Saved chunks to agentic_chunks/agentic_chunks_checkpoint.json

Adding: '"Listening to the Quran can help to memorise and recharge your soul.", '
Api key index: 0
Api key: csk-x2kewhvre2y3cvt859wckjvh66wcn3n3k3nwcn5n4kem8exc
[23/Aug/2025 21:43:38] INFO - HTTP Request: POST https://api.cerebras.ai/v1/chat/completions "HTTP/1.1 200 OK"
[DEBUG] Chunk Found: 7e8ab
Chunk Found (7e8ab), adding to: Qur'an Audio Resources
Api key index: 0
Api key: csk-x2kewhvre2y3cvt859wck

In [4]:
# Get chunks as a dictionary
chunks_dict = ac.get_chunks(get_type='dict')
print(len(chunks_dict))
print(chunks_dict)
# Save Chunk dict to JSON file
with open("agentic_chunks/chunk_dictionary.json", "w") as f:
    json.dump(chunks_dict, f, indent=4)

NameError: name 'ac' is not defined

In [26]:
# Load chunks_dict from JSON file
with open("agentic_chunks/chunk_dictionary.json", "r") as f:
    chunks_dict = json.load(f)
print(chunks_dict)

{'7e8ab': {'chunk_id': '7e8ab', 'propositions': ['["Audio is available to listen to the Quran.", ', '"Listening to the Quran can help to memorise and recharge your soul.", ', '"The app provides Quran recitations by over 50 renowned reciters.", ', '"The reciters include Mishary Al Afasy, Husary (Muallim), Ayman Suwaid, Abdur Rahman As-Sudais, Abdul Basit, and Minshawi.", ', '"Ayman Suwaid and Husary (Muallim) are suitable for improving recitation and tajweed.", ', '"Yasser Salama Hadr has a faster recitation.", ', '"Audio translations are available in English, Bangla, and Urdu.", ', '"More audio translations are planned.", ', '"The app allows downloading audio for offline listening.", ', '"The app allows streaming audio to save storage space.", ', '"The repeat option assists with Quran memorization (Hifz).", ', '"Users can choose from different reciters.", ', '"Users can choose from different audio translations.", ', '  "The Quran Planner helps users break down their reading into manage

In [27]:
import re

# Create a list of Document objects from chunk_dict for embedding
documents = [
    Document(
        page_content=re.sub(r'^[\[\]"\',\s]+|[\[\]"\',\s]+$', '', " ".join(chunk['propositions'])),
        metadata=chunk['metadata'] if 'metadata' in chunk else {
            "chunk_id": chunk['chunk_id'],
            "title": chunk['title'],
            "summary": chunk['summary'],
            "chunk_index": chunk['chunk_index']
        }
    )
    for chunk in chunks_dict.values()
]

print(documents)

[Document(metadata={'chunk_id': '7e8ab', 'title': 'Islamic Digital Quran Resources', 'summary': 'This chunk outlines digital Quran resources—audio recitations, multilingual translations, reading planners, and Mushaf mode—to aid remembrance, understanding, and memorization of the Qur’an in line with Islamic devotion.', 'chunk_index': 0}, page_content='Audio is available to listen to the Quran.",  "Listening to the Quran can help to memorise and recharge your soul.",  "The app provides Quran recitations by over 50 renowned reciters.",  "The reciters include Mishary Al Afasy, Husary (Muallim), Ayman Suwaid, Abdur Rahman As-Sudais, Abdul Basit, and Minshawi.",  "Ayman Suwaid and Husary (Muallim) are suitable for improving recitation and tajweed.",  "Yasser Salama Hadr has a faster recitation.",  "Audio translations are available in English, Bangla, and Urdu.",  "More audio translations are planned.",  "The app allows downloading audio for offline listening.",  "The app allows streaming aud

In [59]:
# save the documents to a json file for debugging
with open("agentic_chunks/documents.json", "w") as f:
    json.dump([{"page_content": doc.page_content, "metadata": doc.metadata} for doc in documents], f, indent=4)

In [7]:
with open("agentic_chunks/documents.json", "r") as f:
    documents_data = json.load(f)
    documents = [Document(page_content=doc["page_content"], metadata=doc["metadata"]) for doc in documents_data]
print(documents)

[Document(metadata={'chunk_id': '7e8ab', 'title': 'Islamic Digital Quran Resources', 'summary': 'This chunk outlines digital Quran resources—audio recitations, multilingual translations, reading planners, and Mushaf mode—to aid remembrance, understanding, and memorization of the Qur’an in line with Islamic devotion.', 'chunk_index': 0}, page_content='Audio is available to listen to the Quran.",  "Listening to the Quran can help to memorise and recharge your soul.",  "The app provides Quran recitations by over 50 renowned reciters.",  "The reciters include Mishary Al Afasy, Husary (Muallim), Ayman Suwaid, Abdur Rahman As-Sudais, Abdul Basit, and Minshawi.",  "Ayman Suwaid and Husary (Muallim) are suitable for improving recitation and tajweed.",  "Yasser Salama Hadr has a faster recitation.",  "Audio translations are available in English, Bangla, and Urdu.",  "More audio translations are planned.",  "The app allows downloading audio for offline listening.",  "The app allows streaming aud

# Chunk Embedding

In [28]:
# Index construction
def build_index(documents: list):
    all_chunks = [doc.page_content for doc in documents]
    metadata = [doc.metadata for doc in documents]

    # Sparse (BM25)
    tokenized_corpus = [chunk.split(" ") for chunk in all_chunks]
    bm25 = BM25Okapi(tokenized_corpus)

    # Dense Embeddings
    embedding_model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2', device=device)
    dense_embeddings = embedding_model.encode(all_chunks, convert_to_numpy=True, normalize_embeddings=True)
    dim = dense_embeddings.shape[1]

    # FAISS (HNSW)
    index = faiss.IndexHNSWFlat(dim, 32)
    index.hnsw.efConstruction = 40
    faiss.normalize_L2(dense_embeddings)
    index.add(dense_embeddings)

    # Chroma
    chroma_db = Chroma.from_texts(
        texts=all_chunks,
        embedding=HuggingFaceEmbeddings(model_name='sentence-transformers/all-mpnet-base-v2'),
        metadatas=metadata,
        persist_directory="./chroma_agentic"
    )

    return bm25, index, dense_embeddings, all_chunks, metadata, chroma_db

In [ ]:
# # Hybrid retrieval with Chroma
# def retrieve_context(query: str, bm25, faiss_index, corpus: List[str], metadata: List[dict], chroma_db, top_k: int = 5, rerank_k: int = 15) -> tuple:
#     cross_encoder = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-6-v2', device=device)
#     embedding_model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2', device=device)

#     # BM25 retrieval
#     tokenized_query = query.split(" ")
#     bm25_scores = bm25.get_scores(tokenized_query)
#     bm25_top_idx = np.argsort(bm25_scores)[::-1][:top_k]

#     # FAISS retrieval
#     query_emb = embedding_model.encode(query, convert_to_numpy=True, normalize_embeddings=True)
#     faiss.normalize_L2(query_emb.reshape(1, -1))
#     _, dense_top_idx = faiss_index.search(query_emb.reshape(1, -1), top_k)

#     # Chroma retrieval
#     chroma_results = chroma_db.similarity_search_with_score(query, k=top_k)
#     chroma_top_idx = [corpus.index(doc.page_content) for doc, _ in chroma_results if doc.page_content in corpus]

#     # Combine and deduplicate
#     candidate_indices = set(bm25_top_idx) | set(dense_top_idx[0]) | set(chroma_top_idx)
#     candidates = [(i, corpus[i], metadata[i]) for i in candidate_indices if i < len(corpus)]

#     # Rerank
#     pairs = [[query, chunk] for _, chunk, _ in candidates]
#     scores = cross_encoder.predict(pairs)
#     reranked = sorted(zip(scores, candidates), key=lambda x: x[0], reverse=True)[:rerank_k]

#     contexts = [chunk for _, (_, chunk, _) in reranked]
#     docs = [meta for _, (_, _, meta) in reranked]

#     # print contexts and metadata
#     # for i, (context, meta) in enumerate(zip(contexts, docs)):
#     #     print(f"Context {i+1}: {context}")
#     #     print(f"Metadata: {meta}")

#     # print all the results of the retrieval techniques
#     # print(f"BM25 Top Indices: {bm25_top_idx}")
#     # print(f"FAISS Top Indices: {dense_top_idx[0]}")
#     # print(f"Chroma Top Indices: {chroma_top_idx}")
#     # print(f"Combined Candidate Indices: {candidate_indices}")
#     # print(f"Reranked Contexts: {contexts}")
#     return contexts, docs

In [29]:
cross_encoder = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-6-v2', device=device)
embedding_model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2', device=device)

[26/Aug/2025 18:26:49] INFO - Load pretrained SentenceTransformer: sentence-transformers/all-mpnet-base-v2


In [30]:
# Build index
bm25, index, dense_embeddings, all_chunks, metadata, chroma_db = build_index(documents)

[26/Aug/2025 18:26:54] INFO - Load pretrained SentenceTransformer: sentence-transformers/all-mpnet-base-v2


Batches: 100%|██████████| 3/3 [00:01<00:00,  1.97it/s]

[26/Aug/2025 18:27:01] INFO - Use pytorch device_name: cuda:0
[26/Aug/2025 18:27:01] INFO - Load pretrained SentenceTransformer: sentence-transformers/all-mpnet-base-v2



/tmp/ipykernel_4725/197068750.py:24: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding=HuggingFaceEmbeddings(model_name='sentence-transformers/all-mpnet-base-v2'),


[26/Aug/2025 18:27:07] INFO - Anonymized telemetry enabled. See                     https://docs.trychroma.com/telemetry for more information.


In [31]:
# Hybrid retrieval with Chroma (multi-query support)
def retrieve_context(queries: list, bm25, faiss_index, corpus: list, metadata: list, chroma_db, top_k: int = 5, rerank_k: int = 5) -> tuple:
    # cross_encoder = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-6-v2', device=device)
    # embedding_model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2', device=device)
    global cross_encoder, embedding_model

    # Collect candidate indices from all queries
    candidate_indices = set()
    for query in queries:
        # BM25 retrieval
        tokenized_query = query.split(" ")
        bm25_scores = bm25.get_scores(tokenized_query)
        bm25_top_idx = np.argsort(bm25_scores)[::-1][:top_k]

        # FAISS retrieval
        query_emb = embedding_model.encode(query, convert_to_numpy=True, normalize_embeddings=True)
        faiss.normalize_L2(query_emb.reshape(1, -1))
        _, dense_top_idx = faiss_index.search(query_emb.reshape(1, -1), top_k)

        # Chroma retrieval
        chroma_results = chroma_db.similarity_search_with_score(query, k=top_k)
        chroma_top_idx = [corpus.index(doc.page_content) for doc, _ in chroma_results if doc.page_content in corpus]

        # Combine indices for this query
        candidate_indices.update(bm25_top_idx)
        candidate_indices.update(dense_top_idx[0])
        candidate_indices.update(chroma_top_idx)

    # Deduplicate and collect candidates
    candidates = [(i, corpus[i], metadata[i]) for i in candidate_indices if i < len(corpus)]

    # Rerank all candidates with all queries
    pairs = []
    for query in queries:
        pairs.extend([[query, chunk] for _, chunk, _ in candidates])
    scores = cross_encoder.predict(pairs)
    # Average scores for each candidate across all queries
    scores_per_candidate = {}
    idx = 0
    for query in queries:
        for i, (cand_idx, _, _) in enumerate(candidates):
            scores_per_candidate.setdefault(cand_idx, []).append(scores[idx])
            idx += 1
    avg_scores = [(np.mean(scores), candidates[i]) for i, (cand_idx, _, _) in enumerate(candidates)]
    reranked = sorted(avg_scores, key=lambda x: x[0], reverse=True)[:rerank_k]

    contexts = [chunk for _, (_, chunk, _) in reranked]
    docs = [meta for _, (_, _, meta) in reranked]
    return contexts, docs

# Query Expansion

In [32]:
def generate_app_documentation_multi_query(query, model="gpt-oss-120b"):
    try:
        prompt = f"""
        You are an expert assistant for Islamic application review analysis.
        Your task is to help users find relevant information in the app documentation by expanding their review or query.
        Given the user's review or question, generate up to five concise, single-topic follow-up questions that explore different aspects of the original query.
        Each question should be directly related to the review and help clarify or expand on the user's needs.
        Do not use compound sentences. List each question on a separate line without numbering.
        Review: {query}
        """
        messages = [
            {
                "role": "system",
                "content": prompt,
            },
            {"role": "user", "content": query},
        ]
        global client, current_api_key_index, api_keys
        response = client.chat.completions.create(
            model=model,
            messages=messages,
        )
        content = response.choices[0].message.content
        content = content.split("\n")
        return content
    except Exception as e:
            error_msg = e.__str__()
            if "request_quota_exceeded" in error_msg:
                logger.error(f"Request quota exceeded for key serial {current_api_key_index}.")
                current_api_key_index += 1
                logger.info(f"Switching to API key serial {current_api_key_index}.")
            elif "token_quota_exceeded" in error_msg:
                logger.error(f"Token quota exceeded for key serial {current_api_key_index}. Please check your API key limits.")
                api_keys.pop(current_api_key_index)
            else:
                logger.error(f"An error occurred: {error_msg}")
                raise e

            if current_api_key_index >= len(api_keys):
                current_api_key_index = 0
            if len(api_keys) == 0:
                raise ValueError("No valid Cerebras API keys available.")
            client = Cerebras(api_key=api_keys[current_api_key_index])


In [33]:
def generate_ettiquete_related_query(model="gpt-oss-120b"):
    try:
        prompt = f"""You are an Islamic etiquette specialist.```Generate 3 questions to retrieve Islamic guidance for```responding to user reviews```Focus on retrieving chunks about:
        - Islamic greetings (Assalamu Alikum, proper responses)
        - Islamic manners and respectful communication 
        - Islamic social behavior and conflict resolution

        Generate questions that will match knowledge base content:```"Islamic etiquette and greetings"```"Islamic manners and social etiquette"```"Islamic greetings and their meanings"``` "Islamic social behavior and etiquette"```"Islamic etiquette in daily life"```Each question on separate line, no numbering."""

        messages = [
            {
                "role": "system",
                "content": prompt,
            }
        ]
        global client, current_api_key_index, api_keys
        response = client.chat.completions.create(
            model=model,
            messages=messages,
        )
        content = response.choices[0].message.content
        content = content.split("\n")
        return content
    except Exception as e:
            error_msg = e.__str__()
            if "request_quota_exceeded" in error_msg:
                logger.error(f"Request quota exceeded for key serial {current_api_key_index}.")
                current_api_key_index += 1
                logger.info(f"Switching to API key serial {current_api_key_index}.")
            elif "token_quota_exceeded" in error_msg:
                logger.error(f"Token quota exceeded for key serial {current_api_key_index}. Please check your API key limits.")
                api_keys.pop(current_api_key_index)
            else:
                logger.error(f"An error occurred: {error_msg}")
                raise e

            if current_api_key_index >= len(api_keys):
                current_api_key_index = 0
            if len(api_keys) == 0:
                raise ValueError("No valid Cerebras API keys available.")
            client = Cerebras(api_key=api_keys[current_api_key_index])


In [34]:
def generate_prompt(review, aug_queries: list, etiquette_data: list, aspects: list, bm25, faiss_index, corpus: list, metadata: list, chroma_db) -> str:
    joint_query = [review] + aug_queries
    contexts, docs = retrieve_context(joint_query, bm25, faiss_index, corpus, metadata, chroma_db)
    etiquette_contexts, etiquette_docs = retrieve_context(etiquette_data, bm25, faiss_index, corpus, metadata, chroma_db)

    # Deduplicate all chunks by (context, doc) pair
    seen = set()
    dedup_contexts, dedup_docs = [], []
    for ctx, doc in list(zip(contexts, docs)) + list(zip(etiquette_contexts, etiquette_docs)):
        key = (ctx, str(doc))
        if key not in seen:
            dedup_contexts.append(ctx)
            dedup_docs.append(doc)
            seen.add(key)

    combined_context = "\n\n".join([f"{doc.get('source', doc.get('title', ''))}:\n{ctx}" for ctx, doc in zip(dedup_contexts, dedup_docs)])
    aspect_lines = [f"{a['aspect']}: {a['polarity']}" for a in aspects]
    prompt = f"User Review:\n" + "\n".join([review]) + f"\n\nExpanded_Queries based on review:\n" + "\n".join(aug_queries) + f"\n\nAspect-Based Sentiments:\n" + "\n".join(aspect_lines) + f"\n\nRetrieved Chunks:\n{combined_context}"
    return prompt


In [ ]:
from helper_utils import project_embeddings, word_wrap
import umap

embeddings = chroma_db.get(include=["embeddings"])["embeddings"]
umap_transform = umap.UMAP(random_state=0, transform_seed=0).fit(embeddings)
projected_dataset_embeddings = project_embeddings(embeddings, umap_transform)

empbedding_model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2', device=device)

# Visualize the results in the embedding space
original_query_embedding = empbedding_model.encode([original_query], convert_to_numpy=True, normalize_embeddings=True)
augmented_query_embeddings = empbedding_model.encode(joint_query, convert_to_numpy=True, normalize_embeddings=True)

project_original_query = project_embeddings(original_query_embedding, umap_transform)
project_augmented_queries = project_embeddings(augmented_query_embeddings, umap_transform)

retrieved_embeddings = dense_embeddings  # Assuming results["embeddings"] refers to dense_embeddings
projected_result_embeddings = project_embeddings(retrieved_embeddings, umap_transform)

import matplotlib.pyplot as plt

# Plot the projected query and retrieved documents in the embedding space
plt.figure()
plt.scatter(
    projected_dataset_embeddings[:, 0],
    projected_dataset_embeddings[:, 1],
    s=10,
    color="gray",
)
plt.scatter(
    project_augmented_queries[:, 0],
    project_augmented_queries[:, 1],
    s=150,
    marker="X",
    color="orange",
)
plt.scatter(
    projected_result_embeddings[:, 0],
    projected_result_embeddings[:, 1],
    s=100,
    facecolors="none",
    edgecolors="g",
)
plt.scatter(
    project_original_query[:, 0],
    project_original_query[:, 1],
    s=150,
    marker="X",
    color="r",
)

plt.gca().set_aspect("equal", "datalim")
plt.title(f"{original_query}")
plt.axis("off")
plt.show()

In [ ]:
import matplotlib.pyplot as plt

# Plot the projected query and retrieved documents in the embedding space
plt.figure(figsize=(12, 8))
plt.scatter(
    projected_dataset_embeddings[:, 0],
    projected_dataset_embeddings[:, 1],
    s=10,
    color="gray",
    label="Dataset Embeddings",
)
plt.scatter(
    project_augmented_queries[:, 0],
    project_augmented_queries[:, 1],
    s=150,
    marker="X",
    color="orange",
    label="Augmented Queries",
)
plt.scatter(
    projected_result_embeddings[:, 0],
    projected_result_embeddings[:, 1],
    s=100,
    facecolors="none",
    edgecolors="g",
    label="Retrieved Data",
)
plt.scatter(
    project_original_query[:, 0],
    project_original_query[:, 1],
    s=150,
    marker="X",
    color="r",
    label="Original Query",
)

# Annotate the queries
for i, (x, y) in enumerate(project_augmented_queries):
    plt.text(x + 0.2, y, joint_query[i], fontsize=8, color="orange")

# Annotate the retrieved data
for i, (x, y) in enumerate(projected_result_embeddings):
    plt.text(x + 0.2, y, f"Retrieved {i+1}", fontsize=8, color="green")

# Add a legend
plt.legend(loc="upper left")

# Add a list of queries and retrieved data on the right side
query_text = "\n".join([f"Query {i+1}: {q}" for i, q in enumerate(joint_query)])
retrieved_text = "\n".join([f"Retrieved {i+1}: {doc.page_content[:350]}..." for i, doc in enumerate(documents)])

plt.gca().set_aspect("equal", "datalim")
plt.title(f"{original_query}")
plt.axis("off")

# Add text box for queries and retrieved data
plt.gcf().text(0.75, 0.5, f"Queries:\n{query_text}\n\nRetrieved Data:\n{retrieved_text}", fontsize=10, va="center", ha="left")

plt.show()

In [ ]:
from pyabsa import AspectTermExtraction as ATEPC
from typing import Generator

aspect_extractor = ATEPC.AspectTermExtractor("english", device=device)

def extract_aspects_from_query(query: str) -> Generator[str, None, None]:
    """
    Extract aspects from a given query using the aspect extractor.
    
    Args:
        query (str): The input query from which to extract aspects.
    
    Returns:
        Generator[str, None, None]: A generator yielding extracted aspects.
    """
    results = aspect_extractor.extract_aspect_terms(query)
    for result in results:
        yield result.aspect_term
        print(f"Extracted Aspect: {result.aspect_term}")  # Print each extracted aspect

# Generate Review Response

In [61]:
def generate_islamic_app_review_response(generation_prompt: str, model="gpt-oss-120b") -> str:
    """
    Generates a polite, empathetic, and focused review response for an Islamic app query,
    using the provided combined string of query and retrieved chunks,
    while tailoring the response based on PyABSA aspects and their sentiment polarity.

    Args:
        generation_prompt (str): The user query plus relevant context chunks.
        aspects (list): List of dicts with 'aspect' and 'polarity' keys from PyABSA.
        model (str): Model identifier.

    Returns:
        str: The generated review response.
    """
    system_prompt = """
        You are an Islamic app support assistant who responds to user reviews with politeness, empathy, and strict adherence to Islamic etiquette (Adab). 

        Start your response with a respectful Islamic greeting such as "Assalamu Alaikum wa Rahmatullahi wa Barakatuh."

        Always express gratitude for the user's feedback (shukr), e.g., "JazakAllahu Khair for sharing your experience."

        Use a gentle, dignified tone (adab al-hiwar) reflecting humility, sincerity, and good character (husn al-khuluq). Even with negative reviews, maintain patience (sabr) and kindness.

        Consider the PyABSA generated aspects with their polarity to tailor your response: 
        - If aspect polarity is positive, express appreciation and reinforce the app’s value.
        - If neutral or feature-request, thank the user and show openness to improvements.
        - If negative, acknowledge the issue calmly, provide a clear step or solution based on context, and reassure the user.

        Focus your response only on the aspects and issues mentioned. Avoid speculation or unrelated topics.

        Structure your reply in two short paragraphs:
        1. Acknowledge the user’s concern with empathy and reassurance.
        2. Provide a clear, step-by-step suggestion or next action to address the issue in short.

        After addressing the content, connect your response to Islamic values such as service to the Ummah (Khidmah), and the noble purpose of the app (e.g., facilitating Ibadah or Quranic knowledge).

        Invite continued engagement with the app and offer support channels if applicable.

        For technical issues, feature requests, or detailed feedback, direct users to submit their feedback at feedback.gtaf.org for proper tracking and response.

        Close with a respectful du’a or salam, such as "May Allah accept from you and us. BarakAllahu feekum."

        Do not promise outcomes you cannot guarantee or speculate about technical causes without evidence.

        Limit discussion to features mentioned in the review.

        ---

        Example opening: "Assalamu Alaikum wa Rahmatullahi wa Barakatuh. JazakAllahu Khair for taking the time to share your feedback."

        Use concise, respectful, and clear language throughout.
        """


    try:
        global client, current_api_key_index, api_keys
        print(f"API KEY in use: {api_keys[current_api_key_index]}\n")
        completion = client.chat.completions.create(
            model=model,
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": generation_prompt}
            ],
            temperature=0.7,
            top_p=0.9,
            reasoning_effort="medium"
        )
        return completion.choices[0].message.content.strip()
    except Exception as e:
        error_msg = e.__str__()
        if "request_quota_exceeded" in error_msg:
            logger.error(f"Request quota exceeded for key serial {current_api_key_index}.")
            current_api_key_index += 1
            logger.info(f"Switching to API key serial {current_api_key_index}.")
        elif "token_quota_exceeded" in error_msg:
            logger.error(f"Token quota exceeded for key serial {current_api_key_index}. Please check your API key limits.")
            api_keys.pop(current_api_key_index)
        else:
            logger.error(f"An error occurred: {error_msg}")
            raise e

        if current_api_key_index >= len(api_keys):
            current_api_key_index = 0
        if len(api_keys) == 0:
            raise ValueError("No valid Cerebras API keys available.")
        client = Cerebras(api_key=api_keys[current_api_key_index])


In [ ]:
# from pyabsa import AspectTermExtraction as ATEPC
# from typing import Generator

# aspect_extractor = ATEPC.Predictor("english")

# def extract_aspects_sentiments(text: str) -> list[tuple[str, str]]:
#     """Extract aspects and sentiments from the given text.

#     Args:
#         text (str): The input text to analyze.

#     Yields:
#         list[tuple[str, str]]: A list of tuple containing the aspect and its corresponding sentiment.

#     Example:
#         >>> sample_text = "The food was great but the service was terrible."
#         >>> for aspect, sentiment in extract_aspects_sentiments(sample_text):
#         ...     print(f"Aspect: {aspect}, Sentiment: {sentiment}")
#         Aspect: food, Sentiment: positive
#         Aspect: service, Sentiment: negative
#     """
#     result = aspect_extractor.predict(text)
#     results = []
#     for aspect, sentiment in zip(result['aspect'], result['sentiment']):
#         results.append((aspect, sentiment))
#     return results

# extracted_aspects = extract_aspects_sentiments(original_query);
# response = extracted_aspects+response


In [ ]:
absa_df = pd.read_csv("../Evaluation/out/absa_results_pyabsa.csv")

filtered_df = absa_df[
    (absa_df['review'].str.len() >= 30) &
    (absa_df['review'].str.len() <= 249) &
    (absa_df['aspects'].notnull()) &
    (absa_df['aspects'] != '[]')
].reset_index(drop=True)

filtered_df = filtered_df.sample(n=6000, random_state=42).reset_index(drop=True)

# add ids to every row, also remove the previous index column if it exists
filtered_df['id'] = filtered_df.index
if 'index' in filtered_df.columns:
    filtered_df = filtered_df.drop(columns=['index'])

# remove if any rows have have any language other than English
import langdetect
def is_english(text):
    try:
        return langdetect.detect(text) == 'en'
    except:
        return False
filtered_df = filtered_df[filtered_df['review'].apply(is_english)].reset_index(drop=True)

5253

In [68]:
filtered_df
# Save filtered_df in a csv file in a directory called filtered_reviews
import os
os.makedirs("filtered_reviews", exist_ok=True)
filtered_df.to_csv("filtered_reviews/filtered_reviews.csv", index=False)

In [ ]:
RESPONSE_GENERATION_CHECKPOINT_FILE = "GPT_OSS/last_index.txt"
OUTPUT_FILE = "GPT_OSS/review_responses.csv"

# Load last index from checkpoint if exists
if os.path.exists(RESPONSE_GENERATION_CHECKPOINT_FILE):
    with open(RESPONSE_GENERATION_CHECKPOINT_FILE, "r") as f:
        last_index = int(f.read().strip())
else:
    last_index = -1  # start from beginning

for idx, row in filtered_df.iloc[last_index+1:].iterrows():  # resume from checkpoint
    try:
        review = row['review'] if 'review' in row else row['text']
        aspects_str = row['aspects']
        aspects = ast.literal_eval(aspects_str)
        
        aug_queries = generate_app_documentation_multi_query(review)
        etiquette_data = generate_ettiquete_related_query()
        
        generation_prompt = generate_prompt(
            review, aug_queries, etiquette_data, aspects,
            bm25, index, all_chunks, metadata, chroma_db
        )
        response = generate_islamic_app_review_response(generation_prompt)
        
        # Ensure directory exists
        os.makedirs(os.path.dirname(OUTPUT_FILE), exist_ok=True)
        # Create file with header if it doesn't exist
        if not os.path.exists(OUTPUT_FILE):
            with open(OUTPUT_FILE, "w") as f:
                f.write("review,aspects,response\n")
        # Save review, aspects and response to csv
        with open(OUTPUT_FILE, "a") as f:
            f.write(f'"{review}","{aspects_str}","{response}"\n')

        # ✅ Update checkpoint after success
        with open(RESPONSE_GENERATION_CHECKPOINT_FILE, "w") as f:
            f.write(str(idx))

    except Exception as e:
        # Save partial data to CSV in case of error
        os.makedirs(os.path.dirname(OUTPUT_FILE), exist_ok=True)
        if not os.path.exists(OUTPUT_FILE):
            with open(OUTPUT_FILE, "w") as f:
                f.write("review,aspects,response\n")
        with open(OUTPUT_FILE, "a") as f:
            f.write(f'"{review}","{aspects_str}","{response}"\n')

        print(f"Error processing row {idx}: {e}")
        error_msg = e.__str__()
        if "request_quota_exceeded" in error_msg:
            logger.error(f"Request quota exceeded for key serial {current_api_key_index}.")
            current_api_key_index += 1
            logger.info(f"Switching to API key serial {current_api_key_index}.")
        elif "token_quota_exceeded" in error_msg:
            logger.error(f"Token quota exceeded for key serial {current_api_key_index}. Please check your API key limits.")
            api_keys.pop(current_api_key_index)
        else:
            logger.error(f"An error occurred: {error_msg}")
            continue

        if current_api_key_index >= len(api_keys):
            current_api_key_index = 0
        if len(api_keys) == 0:
            raise ValueError("No valid Cerebras API keys available.")
        client = Cerebras(api_key=api_keys[current_api_key_index])


In [73]:
import pandas as pd

# Load the review responses CSV file
review_responses_df = pd.read_csv("GPT_OSS/review_responses.csv")
review_responses_df

# print the column names
print(review_responses_df.columns)
# delete the rows which's Unnamed: 3 column is not null
if 'Unnamed: 3' in review_responses_df.columns:
    review_responses_df = review_responses_df[review_responses_df['Unnamed: 3'].isnull()].drop(columns=['Unnamed: 3'])
    review_responses_df = review_responses_df.reset_index(drop=True)
review_responses_df

Index(['review', 'aspects', 'response', 'Unnamed: 3'], dtype='object')


,review,aspects,response
0,its great tools for learning arabic word by word,"[{'aspect': 'tools', 'polarity': 'Positive'}, ...",Assalamu Alaikum wa Rahmatullahi wa Barakatuh....
1,Very wonderful app may Allah reward you more a...,"[{'aspect': 'app', 'polarity': 'Positive'}, {'...",Assalamu Alaikum wa Rahmatullahi wa Barakatuh....
2,A very reliable app with no problems 🤞🤞,"[{'aspect': 'app', 'polarity': 'Positive'}]",Assalamu Alaikum wa Rahmatullahi wa Barakatuh....
3,"Very good qur'an app, has a lot of features","[{'aspect': 'app', 'polarity': 'Positive'}]",Assalamu Alaikum wa Rahmatullahi wa Barakatuh....
4,Such apps are a great help to the Muslim Ummah...,"[{'aspect': 'apps', 'polarity': 'Positive'}]",Assalamu Alaikum wa Rahmatullahi wa Barakatuh....
...,...,...,...
979,I've just started using it. Masha'Allah it loo...,"[{'aspect': 'ads', 'polarity': 'Positive'}]",Assalamu Alaikum wa Rahmatullahi wa Barakatuh....
980,Very good.I like this app too much,"[{'aspect': 'app', 'polarity': 'Positive'}]",Assalamu Alaikum wa Rahmatullahi wa Barakatuh....
981,This is the best app for Quran that I used so ...,"[{'aspect': 'app', 'polarity': 'Positive'}, {'...",Assalamu Alaikum wa Rahmatullahi wa Barakatuh....
982,Alhumdulilla this app is very good and easy-to...,"[{'aspect': 'app', 'polarity': 'Positive'}]",Assalamu Alaikum wa Rahmatullahi wa Barakatuh....


In [74]:
# Save to GPT_OSS/review_responses.csv
review_responses_df.to_csv("GPT_OSS/review_responses.csv", index=False)